In [1]:
%load_ext cython

In [20]:
import math


def py_trapz(f, a, b, n):
    """
    Numerically integrates a function f over the interval [a, b] using the trapezoidal rule.

    Parameters:
        f (function): The function to integrate.
        a (float): The start of the interval.
        b (float): The end of the interval.
        n (int or float): The number of subintervals for the trapezoidal approximation.

    Returns:
        float: The approximate integral of the function over the interval [a, b].
    """
    n = int(n)  # Convert n to an integer to ensure the number of subintervals is valid
    inter = (b - a) / n  # Calculate the width of each subinterval
    sum = 0.5 * (f(a) + f(b))  # Start with half the first and last terms

    # Sum the interior terms
    for i in range(1, n):
        sum += f(a + i * inter)

    return (
        sum * inter
    )  # Multiply by the width of the subintervals to get the final integral approximation


def square(x):
    """
    Returns the square of a number.

    Parameters:
        x (float): The input number.

    Returns:
        float: The square of x.
    """
    return x * x


def expo(x):
    """
    Returns the exponential of a number.

    Parameters:
        x (float): The input number.

    Returns:
        float: The exponential of x.
    """
    return math.exp(x)


def inv(x):
    """
    Returns the multiplicative inverse of a number.

    Parameters:
        x (float): The input number.

    Returns:
        float: The inverse of x.
    """
    return 1 / x


def sinus(x):
    return math.sin(x)


# Measure the execution time of the integration using the square function over the interval [1, 10] with 1e7 subintervals
%timeit py_trapz(square, 1, 10, 1e7)

# Print the result of the integration
print(py_trapz(square, 1, 10, 1e7))

1.24 s ± 31.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
333.00000000000125


In [21]:
%%cython --annotate
cimport libc.math as math  # Import C standard library for math functions
cimport cython
from libc.stdio cimport printf, fflush, stdout  # Import C standard library for I/O functions

# Disable bounds checking and wraparound for performance
@cython.boundscheck(False)  # Turn off bounds checking for array access
@cython.wraparound(False)   # Turn off negative index wraparound for arrays
@cython.cdivision(True)     # Allow division by zero (C-style division)

cdef double cy_trapz(double (*f)(double), double a, double b, long n):
    """
    Numerically integrates a C function f over the interval [a, b] using the trapezoidal rule.
    
    Parameters:
        f (C function pointer): The function to integrate.
        a (double): The start of the interval.
        b (double): The end of the interval.
        n (long): The number of subintervals for the trapezoidal approximation.
        
    Returns:
        double: The approximate integral of the function over the interval [a, b].
    """
    cdef double inter = (b - a) / n  # Calculate the width of each subinterval
    cdef double sum = 0.5 * (f(a) + f(b))  # Start with half the first and last terms
    cdef long i  # Loop variable for iteration

    # Sum the interior terms using the trapezoidal rule
    for i in range(1, n):
        sum += f(a + i * inter)  # Add the function value at each subinterval

    return sum * inter  # Multiply by the width of each interval to get the final integral approximation

# Example function definitions
cdef double square(double x):
    """
    Returns the square of a number.
    
    Parameters:
        x (double): The input number.
        
    Returns:
        double: The square of x.
    """
    return x * x

@cython.boundscheck(False)  # Turn off bounds checking for performance
@cython.wraparound(False)   # Turn off negative index wraparound for performance
cdef double sinus(double x):
    """
    Returns the sine of a number.
    
    Parameters:
        x (double): The input number.
        
    Returns:
        double: The sine of x.
    """
    return math.sin(x)

@cython.boundscheck(False)  # Turn off bounds checking for performance
@cython.wraparound(False)   # Turn off negative index wraparound for performance
cdef double expo(double x):
    """
    Returns the exponential of a number.
    
    Parameters:
        x (double): The input number.
        
    Returns:
        double: The exponential of x.
    """
    return math.exp(x)

@cython.cdivision(True)
@cython.boundscheck(False)  # Turn off bounds checking for performance
@cython.wraparound(False)   # Turn off negative index wraparound for performance
cdef double inv(double x):
    """
    Returns the multiplicative inverse of a number.
    
    Parameters:
        x (double): The input number.
        
    Returns:
        double: The inverse of x.
    """
    return 1.0 / x

# Perform numerical integration on various functions and print the results
printf("%f\n", cy_trapz(square, 1, 10, 10000000))  # Integrate square function over [1, 10] with 10 million subintervals
fflush(stdout)  # Flush the output buffer

printf("%f\n", cy_trapz(square, 0, 1, 10000000))  # Integrate square function over [0, 1] with 10 million subintervals
fflush(stdout)  # Flush the output buffer

printf("%f\n", cy_trapz(sinus, 0, math.pi, 10000000))  # Integrate sine function over [0, π] with 10 million subintervals
fflush(stdout)  # Flush the output buffer

printf("%f\n", cy_trapz(expo, 0, 1, 10000000))  # Integrate exponential function over [0, 1] with 10 million subintervals
fflush(stdout)  # Flush the output buffer

printf("%f\n", cy_trapz(inv, 1, 2, 10000000))  # Integrate inverse function over [1, 2] with 10 million subintervals
fflush(stdout)  # Flush the output buffer


In [22]:
%%timeit
%%cython
cimport libc.math as math  # Import C standard library for math functions
cimport cython
from libc.stdio cimport printf, fflush, stdout  # Import C standard library for I/O functions

# Enable profiling, and disable bounds checking and wraparound for performance
@cython.profile(True)      # Enable Cython profiling for performance analysis
@cython.boundscheck(False) # Turn off bounds checking for array access
@cython.wraparound(False)  # Turn off negative index wraparound for arrays
@cython.cdivision(True)    # Allow division by zero (C-style division)

def cy_trapz(f, float a, float b, int n):
    """
    Numerically integrates a Python function f over the interval [a, b] using the trapezoidal rule.
    
    Parameters:
        f (Python function): The function to integrate.
        a (float): The start of the interval.
        b (float): The end of the interval.
        n (int): The number of subintervals for the trapezoidal approximation.
        
    Returns:
        float: The approximate integral of the function over the interval [a, b].
    """
    cdef double inter = (b - a) / n  # Calculate the width of each subinterval
    cdef double sum = 0.5 * (f(a) + f(b))  # Start with half the first and last terms
    cdef long i  # Loop variable for iteration

    # Sum the interior terms using the trapezoidal rule
    for i in range(1, n):
        sum += f(a + i * inter)  # Add the function value at each subinterval

    return sum * inter  # Multiply by the width of each interval to get the final integral approximation

# Example function definition
def square(double x):
    """
    Returns the square of a number.
    
    Parameters:
        x (double): The input number.
        
    Returns:
        double: The square of x.
    """
    return x * x

# Perform numerical integration on the square function over the interval [1, 10] with 10 million subintervals
cy_trapz(square, 1, 10, 10000000)

617 µs ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
print(cy_trapz(square, 1, 10, 10000000))

332.9999749903578


In [24]:
%%timeit
%%cython
cimport libc.math as math  # Import the C standard library for math functions
cimport cython
from libc.stdio cimport printf, fflush, stdout  # Import C standard library functions for I/O
@cython.boundscheck(False) # Turn off bounds checking for array access to improve performance
@cython.wraparound(False)  # Turn off negative index wraparound for array access
@cython.cdivision(True)    # Enable C-style division to allow division by zero without raising exceptions

def cy_trapz(f, float a, float b, int n):
    """
    Numerically integrates a given Python function f over the interval [a, b] using the trapezoidal rule.
    
    Parameters:
        f (function): The function to be integrated, which takes a float and returns a float.
        a (float): The start of the integration interval.
        b (float): The end of the integration interval.
        n (int): The number of subintervals to use in the approximation. Higher values yield more accurate results.
        
    Returns:
        float: The approximate integral of the function over the specified interval.
    """
    cdef double inter = (b - a) / n  # Calculate the width of each subinterval
    cdef double sum = 0.5 * (f(a) + f(b))  # Initialize the sum with half of the first and last terms
    cdef long i  # Variable for loop iteration

    # Perform the summation of the function values at each interior point
    for i in range(1, n):
        sum += f(a + i * inter)  # Evaluate the function at each subinterval and add to the sum

    # Return the final result, which is the sum multiplied by the width of the subintervals
    return sum * inter  

# Define an example function to be used for numerical integration
def square(double x):
    """
    Computes the square of a given number.
    
    Parameters:
        x (double): The input value to be squared.
        
    Returns:
        double: The square of the input value (x^2).
    """
    return x * x

# Example usage: numerically integrate the square function over the interval [1, 10]
# using 10 million subintervals to approximate the integral.
cy_trapz(square, 1, 10, 10000000)


619 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
%timeit py_trapz(square, 0, 1, 10000000 )

904 ms ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit cy_trapz(square, 0, 1, 10000000)

410 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
import numpy


def impl_num(f, a, b, n):
    x = numpy.linspace(a, b, n)
    y = f(x)
    return numpy.trapz(y, x, n)


def square(x):
    return x * x


%timeit impl_num(square, 0, 1, 10000000)
%timeit impl_num(square, 1, 10, 10000000)

174 ms ± 2.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
174 ms ± 3.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
print(py_trapz(square, 0, 1, 10000000))

0.33333333333335957


In [29]:
print(cy_trapz(square, 0, 1, 10000000))

0.3333333450194629


In [30]:
print(impl_num(square, 0, 1, 10000000))

0.33333333333333504


In [31]:
import math

%timeit py_trapz(sinus, 0, math.pi, 10000000)

1.85 s ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
print(py_trapz(sinus, 0, math.pi, 10000000))

1.9999999999998086


In [34]:
%%cython
# Cython implementation of sin(x)
import cython
cimport libc.math as math
@cython.boundscheck(False)  # Turn off bounds checking for performance
@cython.wraparound(False)   # Turn off negative index wraparound for performance
def sinus(float x):
    return math.sin(x)

In [35]:
%timeit cy_trapz(sinus, 0, math.pi, 10000000)

428 ms ± 4.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
print(cy_trapz(sinus, 0, math.pi, 10000000))

2.0000000000002234


In [37]:
# Numpy implementation of sin(x)
import math

x = math.pi


def sin(x):
    return numpy.sin(x)


%timeit impl_num(sin, 0, x, 10000000)

255 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
print(impl_num(sin, 0, x, 10000000))

1.999999999999982


In [39]:
%timeit py_trapz(expo, 0, 1, 10000000)

1.79 s ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
print(py_trapz(expo, 0, 1, 10000000))

1.7182818284589751


In [42]:
%%cython
# Cython implementation of e^x
import cython
cimport libc.math as math
@cython.boundscheck(False)  # Turn off bounds checking for performance
@cython.wraparound(False)   # Turn off negative index wraparound for performance
def expo(float x):
    return math.exp(x)

In [44]:
%timeit cy_trapz(expo, 0, 1, 10000000)

413 ms ± 2.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
print(cy_trapz(expo, 0, 1, 10000000))

1.7182818602252599


In [46]:
# numpy implementation of e^x
import math


def expo(x):
    return numpy.exp(x)


%timeit impl_num(expo, 0, 1, 10000000)

210 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
print(impl_num(expo, 0, 1, 10000000))

1.718281828459046


In [48]:
%timeit py_trapz(inv, 1, 2, 10000000)

904 ms ± 43.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
print(py_trapz(inv, 1, 2, 10000000))

0.693147180559984


In [50]:
%%cython
# cython implementation of 1/x
import cython
cimport libc.math as math
@cython.cdivision(True)
@cython.boundscheck(False)  # Turn off bounds checking for performance
@cython.wraparound(False)   # Turn off negative index wraparound for performance
def inv(float x):
    return 1.0 / x

In [51]:
%timeit cy_trapz(inv, 1, 2, 10000000)

428 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
print(cy_trapz(inv, 1, 2, 10000000))

0.6931471864031273


In [55]:
# numpy implementation of the integration of 1/x
def num_inv(a, b, n):
    x = numpy.linspace(a, b, n)
    y = 1 / x
    return numpy.trapz(y, x, n)


%timeit num_inv(1,2,10000000)

162 ms ± 2.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
print(num_inv(1, 2, 10000000))

0.6931471805599465
